## Modeling of Exitron Usage in ALS Patients

In [28]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import re

notes:

for each tissue we'd want to use logistic regression to test the significance of how often we use the exitron, on whether the person has or doesnt have ALS

The covariates we'd want to include in the analysis differ by tissue, and are represented here (where disease is cov$Subject.Group=="ALS")
```
design_list <- list("Cerebellum" = ~Disease + Prep + MEAN_READ_LENGTH + Site.Specimen.Collected,
                    "Spinal_Cord_Lumbar" = ~Disease + Site.Specimen.Collected + Prep + PCT_CHIMERAS, 
                    "Spinal_Cord_Cervical" = ~Disease + Prep + MEAN_READ_LENGTH + Site.Specimen.Collected, 
                    "Spinal_Cord_Thoracic" = ~Disease + Prep + MEDIAN_5PRIME_BIAS + PCT_CHIMERAS, 
                    "Cortex_Frontal" = ~Disease + Prep + MEAN_READ_LENGTH + PCT_CHIMERAS, 
                    "Hippocampus" = ~Disease + MEDIAN_5PRIME_BIAS + MEDIAN_3PRIME_BIAS + PCT_CODING_BASES,
                    "Cortex_Motor" = ~Disease + Prep + MEAN_READ_LENGTH + PCT_CHIMERAS, 
                    "Cortex_Temporal" = ~Disease + Prep + PCT_CHIMERAS + MEDIAN_5PRIME_BIAS
                    )
```
Preprocessing the covariates:
```
covs = covs[order(covs$RIN,decreasing = T),]
covs = covs[order(covs$Platform,decreasing = T),]
covs$Site.Specimen.Collected = str_replace_all(covs$Site.Specimen.Collected," ","_")
covs$Prep = str_replace_all(covs$Prep," ","_")
```

And then for each tissue, i.e Spinal_Cord_Cervical , loop through each exitron and do a logistic regression (if within this subset we also have at least 10 people with a nonzero value)
```
TISSUE = "Spinal_Cord_Cervical"
tmp = covs[which(covs$Sample.Source==TISSUE),]
tmp = tmp[which(!duplicated(tmp$ExternalSubjectId)),]
tmp = tmp[,c("RNA","Subject.Group","Prep","MEAN_READ_LENGTH","Site.Specimen.Collected")]
tmp$Subject.Group = tmp$Subject.Group=="ALS"
tmp$exitron_norm = NA
all_res = list()
for (exitron in colnames(norm)) {
    tmp$exitron_norm = norm[tmp$RNA,exitron]
    if (sum(tmp$splice>0)>10) { # At least 10 peeps with a nonzero value
        fit = glm(Subject.Group ~ exitron_norm + Site.Specimen.Collected + Prep + MEAN_READ_LENGTH,family=binomial(link='logit'),data=tmp)
        res = coef(summary(fit))
        all_res[[exitron]] = cbind.data.frame(tissue=TISSUE,exitron=exitron,t(res[2,]))
    }
}
```

In [8]:
# covariates
covs = pd.read_csv("/gpfs/commons/projects/ALS_Consortium_analysis/compbio/data/metadata_updated_sep4_2024_with_merged_mc.csv")
covs.head()


,Unnamed: 0,RNA,Quote,ExternalSubjectId,NeuroBankID,Project,Site.Specimen.Collected,Sex,Ethnicity,Subject.Group,Subject.Group.Subcategory,Family.History.of.ALS.FTD.,MND.with.FTD.,MND.with.Dementia.,Site.of.Motor.Onset,Site.of.Motor.Onset.Detail,Age.at.Symptom.Onset,Age.at.Death,Sample.Source,RIN,Disease.Duration.in.Months,Post.Mortem.Interval.in.Hours,Reported.Genomic.Mutations,C9orf72.Repeat.Expansion..Data.from.CUMC.,ATXN2.Repeat.Expansion..Data.from.CUMC.,Cause.of.Death,Revised.El.Escorial.Criteria,Comorbidities,Prep,Platform,Sex.Call,Sex.Genotype,pct_african,pct_south_asian,pct_east_asian,pct_european,pct_americas,C9.repeat.size,Expanded....30,ATXN2.repeat.size,Intermediate..30.33,DNA,predicted_ancestry,ancestry_pc1,ancestry_pc2,ancestry_pc3,ancestry_pc4,Total Sequences,Sequence length,%GC,total_deduplicated_percentage,avg_sequence_length,per_tile_sequence_quality,per_base_sequence_content,per_sequence_gc_content,per_base_n_content,sequence_duplication_levels,overrepresented_sequences,Total,Assigned,Unassigned_Unmapped,Unassigned_MultiMapping,Unassigned_NoFeatures,Unassigned_Ambiguity,percent_assigned,TOTAL_READS,PF_READS,PCT_PF_READS,PF_READS_ALIGNED,PCT_PF_READS_ALIGNED,PF_ALIGNED_BASES,PF_HQ_ALIGNED_READS,PF_HQ_ALIGNED_BASES,PF_HQ_ALIGNED_Q20_BASES,PF_MISMATCH_RATE,PF_HQ_ERROR_RATE,PF_INDEL_RATE,MEAN_READ_LENGTH,READS_ALIGNED_IN_PAIRS,PCT_READS_ALIGNED_IN_PAIRS,PF_READS_IMPROPER_PAIRS,PCT_PF_READS_IMPROPER_PAIRS,STRAND_BALANCE,PCT_CHIMERAS,PCT_ADAPTER,UNPAIRED_READS_EXAMINED,READ_PAIRS_EXAMINED,SECONDARY_OR_SUPPLEMENTARY_RDS,UNMAPPED_READS,UNPAIRED_READ_DUPLICATES,READ_PAIR_DUPLICATES,READ_PAIR_OPTICAL_DUPLICATES,PERCENT_DUPLICATION,ESTIMATED_LIBRARY_SIZE,TOTAL_CLUSTERS,ALIGNED_READS,AT_DROPOUT,GC_DROPOUT,GC_NC_0_19,GC_NC_20_39,GC_NC_40_59,GC_NC_60_79,GC_NC_80_100,MEDIAN_INSERT_SIZE,MODE_INSERT_SIZE,MEDIAN_ABSOLUTE_DEVIATION,MIN_INSERT_SIZE,MAX_INSERT_SIZE,MEAN_INSERT_SIZE,STANDARD_DEVIATION,READ_PAIRS,WIDTH_OF_10_PERCENT,WIDTH_OF_20_PERCENT,WIDTH_OF_30_PERCENT,WIDTH_OF_40_PERCENT,WIDTH_OF_50_PERCENT,WIDTH_OF_60_PERCENT,WIDTH_OF_70_PERCENT,WIDTH_OF_80_PERCENT,WIDTH_OF_90_PERCENT,WIDTH_OF_95_PERCENT,WIDTH_OF_99_PERCENT,PF_BASES,RIBOSOMAL_BASES,CODING_BASES,UTR_BASES,INTRONIC_BASES,INTERGENIC_BASES,NUM_R1_TRANSCRIPT_STRAND_READS,NUM_R2_TRANSCRIPT_STRAND_READS,NUM_UNEXPLAINED_READS,PCT_R1_TRANSCRIPT_STRAND_READS,PCT_R2_TRANSCRIPT_STRAND_READS,PCT_RIBOSOMAL_BASES,PCT_CODING_BASES,PCT_UTR_BASES,PCT_INTRONIC_BASES,PCT_INTERGENIC_BASES,PCT_MRNA_BASES,PCT_USABLE_BASES,MEDIAN_CV_COVERAGE,MEDIAN_5PRIME_BIAS,MEDIAN_3PRIME_BIAS,MEDIAN_5PRIME_TO_3PRIME_BIAS,PF_NOT_ALIGNED_BASES,c9orf72
0,CGND-HRA-00013,CGND-HRA-00013,CGND_12616,NEUEL133AK6,NEUEL133AK6,Target ALS,University of California San Diego,Female,Not Hispanic/Latino,ALS,Classical/Typical ALS,Unknown,No,No,Limb,"Limb - Lower Limb, Left Leg",59.0,60.0,Cortex_Frontal,6.5,21.0,5.0,Negative for C9orf72 Repeat Expansion,No,Intermediate,ALS,Unknown,"Asthenia, Basal Cell Skin Cancer, Bipolar Di...",Manual KAPA Total,HiSeq 2500,female,XX,0.021888,0.047047,0.00001,0.523282,0.407774,NaN,NaN,NaN,NaN,CGND-HDA-00188-b38,AMR,2.2377,6.3255,-8.2037,0.0844,115841697,36-125,44,66.658071,121.570197,pass,fail,warn,pass,warn,pass,57920848,31704861,1211002,5484066,19520919,0,54.738254,109091758,102772674,0.942076,101483244,0.987454,12507929089,98784260,12180324391,11645508079,0.002806,0.002766,0.000150,122.592974,101404574,0.999225,78670,0.000775,0.499996,0.004342,0.000000,116553,53218324,6749939,2538557,43569,4215186,1469,0.079528,318065855,57921218,113303140,8.452018,0.278805,0.522768,0.856961,0.982295,2.366953,19.217869,327,268,82,19,2216550,334.401019,132.343537,49003138,33,65,95,129,165,207,269,463,6793,19439,102939,12692260922,110047,2899774840,3168828708,5613521761,825693733,197714,18980246,245739,1.0309,98.9691,0.0009,23.1835,25.3346,44.8797,6.6014,48.5181,47.8134,0.319163,0.565950,0.865137,0.722390,184331833,No
1,CGND-HRA-00015,CGND-HRA-00015,CGND_12616,NEUZU200WEQ,NEUZU200WEQ,Target ALS,U

In [7]:
# normalized exitron data 
exitron_data_npy = np.load("normalized_data.npy")
exitron_data = pd.DataFrame(exitron_data_npy)
exitron_data.head()

,0,1,2,3,4,5,6,7,8,9,...,1866,1867,1868,1869,1870,1871,1872,1873,1874,1875
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.541985,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.208333,0.0,0.0


In [12]:
reference_labels = pd.read_parquet("filtered_exitron_data.parquet")
reference_labels.head()


sourceID                     CGND-HRA-00013  CGND-HRA-00015  CGND-HRA-00017  \
title                                                                         
chr10:119042185:119042215:-             0.0             0.0             0.0   
chr10:119042185:119042245:-             0.0             0.0             0.0   
chr10:119042185:119042275:-             0.0             0.0             0.0   
chr10:119042190:119042373:-             0.0             0.0             0.0   
chr10:119042215:119042638:-             0.0             0.0             0.0   

sourceID                     CGND-HRA-00019  CGND-HRA-00020-2  CGND-HRA-00021  \
title                                                                           
chr10:119042185:119042215:-             0.0               0.0             0.0   
chr10:119042185:119042245:-             0.0               0.0             0.0   
chr10:119042185:119042275:-             0.0               0.0             0.0   
chr10:119042190:119042373:-             0.0               0.0             0.0   
chr10:119042215:119042638:-             0.0               0.0             0.0   

sourceID                     CGND-HRA-00023  CGND-HRA-00024  CGND-HRA-00025  \
title                                                                         
chr10:119042185:119042215:-             0.0             0.0             0.0   
chr10:119042185:119042245:-             0.0             0.0             0.0   
chr10:119042185:119042275:-             0.0             0.0             0.0   
chr10:119042190:119042373:-             0.0             0.0             0.0   
chr10:119042215:119042638:-             0.0             0.0             0.0   

sourceID                     CGND-HRA-00026  CGND-HRA-00027  CGND-HRA-00029  \
title                                                                         
chr10:119042185:119042215:-             0.0             0.0             0.0   
chr10:119042185:119042245:-             0.0             0.0             0.0   
chr10:119042185:119042275:-             0.0             0.0             0.0   
chr10:119042190:119042373:-             0.0             0.0             0.0   
chr10:119042215:119042638:-             0.0             0.0             0.0   

sourceID                     CGND-HRA-00031  CGND-HRA-00034  CGND-HRA-00035  \
title                                                                         
chr10:119042185:119042215:-             0.0             0.0             0.0   
chr10:119042185:119042245:-             0.0             0.0             0.0   
chr10:119042185:119042275:-             0.0             0.0             0.0   
chr10:119042190:119042373:-             0.0             0.0             0.0   
chr10:119042215:119042638:-             0.0             0.0             0.0   

sourceID                     CGND-HRA-00037  CGND-HRA-00038  CGND-HRA-00039  \
title                                                                         
chr10:119042185:119042215:-             0.0             0.0             0.0   
chr10:119042185:119042245:-             0.0             0.0             0.0   
chr10:119042185:119042275:-             0.0             0.0             0.0   
chr10:119042190:119042373:-             0.0             0.0             0.0   
chr10:119042215:119042638:-             0.0             0.0             0.0   

sourceID                     CGND-HRA-00040  CGND-HRA-00041  CGND-HRA-00043  \
title                                                                         
chr10:119042185:119042215:-             0.0             0.0             0.0   
chr10:119042185:119042245:-             0.0             0.0             0.0   
chr10:119042185:119042275:-             0.0             0.0             0.0   
chr10:119042190:119042373:-             0.0             0.0             0.0   
chr10:119042215:119042638:-             0.0             0.0             0.0   

sourceID                     CGND-HRA-00045  CGND-HRA-00047  CGND-HRA-00048  \
title                        

In [29]:
# covariates data
covs = pd.read_csv("/gpfs/commons/projects/ALS_Consortium_analysis/compbio/data/metadata_updated_sep4_2024_with_merged_mc.csv")
covs = covs.sort_values(by=['RIN', 'Platform'], ascending=[False, False])
covs['Site.Specimen.Collected'] = covs['Site.Specimen.Collected'].str.replace(' ', '_')
covs['Prep'] = covs['Prep'].str.replace(' ', '_')

# create  binary 'Disease' variable (1 for ALS, 0 for control)
covs['Disease'] = (covs['Subject.Group'] == 'ALS').astype(int)

# exitron data
exitron_data_npy = np.load("normalized_data.npy")
exitron_data = pd.DataFrame(exitron_data_npy) 
# rename columns and rows corresponding with exitron_data/.parquet
reference_labels = pd.read_parquet("filtered_exitron_data.parquet")
exitron_data.columns = reference_labels.columns
exitron_data.index = reference_labels.index

# covariates for each tissue
design_list = {
    "Cerebellum": "~Disease + Prep + MEAN_READ_LENGTH + Site.Specimen.Collected",
    "Spinal_Cord_Lumbar": "~Disease + Site.Specimen.Collected + Prep + PCT_CHIMERAS",
    "Spinal_Cord_Cervical": "~Disease + Prep + MEAN_READ_LENGTH + Site.Specimen.Collected",
    "Spinal_Cord_Thoracic": "~Disease + Prep + MEDIAN_5PRIME_BIAS + PCT_CHIMERAS",
    "Cortex_Frontal": "~Disease + Prep + MEAN_READ_LENGTH + PCT_CHIMERAS",
    "Hippocampus": "~Disease + MEDIAN_5PRIME_BIAS + MEDIAN_3PRIME_BIAS + PCT_CODING_BASES",
    "Cortex_Motor": "~Disease + Prep + MEAN_READ_LENGTH + PCT_CHIMERAS",
    "Cortex_Temporal": "~Disease + Prep + PCT_CHIMERAS + MEDIAN_5PRIME_BIAS"
}

In [34]:
all_results_list = []

for tissue, design in design_list.items():
    print(f"Processing tissue: {tissue}")
    
    # filter data for the specified tissue
    tmp = covs[covs['Sample.Source'] == tissue].copy()
    tmp = tmp.drop_duplicates(subset=['ExternalSubjectId'])
    model_vars = design.split('~')[1].strip().split(' + ')
    columns_to_select = ['RNA', 'Subject.Group'] + model_vars
    if 'Disease' in columns_to_select:
        columns_to_select.remove('Disease')
    tmp = tmp[columns_to_select]
    tmp['Subject.Group'] = (tmp['Subject.Group'] == 'ALS').astype(int)

    # model formula
    predictor_formula = " + ".join(model_vars).replace('Disease', '')
    predictor_formula = re.sub(r'\s*\+\s*\+', '+', predictor_formula.strip()).strip(' +')
    final_model_formula = f"Subject_Group ~ exitron_norm + {predictor_formula}"

    # loop through each exitron 
    for exitron in exitron_data.index:
        # match exitron data by 'RNA'
        exitron_values_for_mapping = exitron_data.loc[exitron]
        tmp['exitron_norm'] = tmp['RNA'].map(exitron_values_for_mapping)        

        if (tmp['exitron_norm'] != 0).sum() > 10: # At least 10 peeps with a nonzero value
            clean_tmp = tmp.rename(columns=lambda c: re.sub(r'[.]', '_', c))
            clean_formula = re.sub(r'[.]', '_', final_model_formula)

            try:
                # fit logistic regression model
                fit = smf.logit(formula=clean_formula, data=clean_tmp).fit(disp=0) # disp=0 hides convergence output
                
                # extract results for the 'exitron_norm' coefficient
                res = fit.summary2().tables[1]
                exitron_res = res.loc['exitron_norm'].rename(fit.params.index[1])
                
                # add tissue and exitron info and append to our results list
                result_row = pd.DataFrame([exitron_res])
                result_row['tissue'] = tissue
                result_row['exitron'] = exitron
                all_results_list.append(result_row)

            except Exception as e:
                # print(f"    Could not fit model for {exitron} in {tissue}. Error: {e}")
                continue

# Combine all results into a single DataFrame
if all_results_list:
    results_df = pd.concat(all_results_list, ignore_index=True)
else:
    print("No results")

/gpfs/commons/home/ncui/miniconda3/envs/vscode/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/gpfs/commons/home/ncui/miniconda3/envs/vscode/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/gpfs/commons/home/ncui/miniconda3/envs/vscode/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/gpfs/commons/home/ncui/miniconda3/envs/vscode/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Like

Processing tissue: Spinal_Cord_Thoracic


/gpfs/commons/home/ncui/miniconda3/envs/vscode/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/gpfs/commons/home/ncui/miniconda3/envs/vscode/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/gpfs/commons/home/ncui/miniconda3/envs/vscode/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/gpfs/commons/home/ncui/miniconda3/envs/vscode/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Like

Processing tissue: Cortex_Frontal


/gpfs/commons/home/ncui/miniconda3/envs/vscode/lib/python3.13/site-packages/statsmodels/discrete/discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/gpfs/commons/home/ncui/miniconda3/envs/vscode/lib/python3.13/site-packages/statsmodels/discrete/discrete_model.py:2443: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q * linpred)))
/gpfs/commons/home/ncui/miniconda3/envs/vscode/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/gpfs/commons/home/ncui/miniconda3/envs/vscode/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Processing tissue: Hippocampus


/gpfs/commons/home/ncui/miniconda3/envs/vscode/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/gpfs/commons/home/ncui/miniconda3/envs/vscode/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/gpfs/commons/home/ncui/miniconda3/envs/vscode/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/gpfs/commons/home/ncui/miniconda3/envs/vscode/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Like

Processing tissue: Cortex_Motor


/gpfs/commons/home/ncui/miniconda3/envs/vscode/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/gpfs/commons/home/ncui/miniconda3/envs/vscode/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/gpfs/commons/home/ncui/miniconda3/envs/vscode/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/gpfs/commons/home/ncui/miniconda3/envs/vscode/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Like

Processing tissue: Cortex_Temporal

--- Final Combined Results ---
         Coef.  Std.Err.         z     P>|z|     [0.025     0.975]  \
0    -0.723267  1.185449 -0.610121  0.541782  -3.046703   1.600170   
1    -0.290065  0.610802 -0.474893  0.634864  -1.487216   0.907085   
2     4.406686  5.193176  0.848553  0.396130  -5.771751  14.585123   
3    -4.033948  3.052461 -1.321540  0.186322 -10.016662   1.948766   
4    -0.650334  1.042348 -0.623913  0.532685  -2.693298   1.392630   
...        ...       ...       ...       ...        ...        ...   
7748  2.194986  4.142583  0.529859  0.596210  -5.924327  10.314299   
7749  1.202727  2.207991  0.544715  0.585949  -3.124856   5.530310   
7750  6.581783  5.250245  1.253614  0.209982  -3.708509  16.872075   
7751 -2.165554  2.270092 -0.953950  0.340109  -6.614853   2.283744   
7752 -6.093024  3.848505 -1.583218  0.113372 -13.635956   1.449907   

               tissue                      exitron  
0          Cerebellum  chr10:119042185:

In [35]:
print(results_df)

         Coef.  Std.Err.         z     P>|z|     [0.025     0.975]  \
0    -0.723267  1.185449 -0.610121  0.541782  -3.046703   1.600170   
1    -0.290065  0.610802 -0.474893  0.634864  -1.487216   0.907085   
2     4.406686  5.193176  0.848553  0.396130  -5.771751  14.585123   
3    -4.033948  3.052461 -1.321540  0.186322 -10.016662   1.948766   
4    -0.650334  1.042348 -0.623913  0.532685  -2.693298   1.392630   
...        ...       ...       ...       ...        ...        ...   
7748  2.194986  4.142583  0.529859  0.596210  -5.924327  10.314299   
7749  1.202727  2.207991  0.544715  0.585949  -3.124856   5.530310   
7750  6.581783  5.250245  1.253614  0.209982  -3.708509  16.872075   
7751 -2.165554  2.270092 -0.953950  0.340109  -6.614853   2.283744   
7752 -6.093024  3.848505 -1.583218  0.113372 -13.635956   1.449907   

               tissue                      exitron  
0          Cerebellum  chr10:119042185:119042245:-  
1          Cerebellum  chr10:133285213:133285455:-  


In [38]:
significant_results = results_df[results_df['P>|z|'] < 0.05]
significant_results.head()
len(significant_results)

853